Deep Learning Models -- A collection of various deep learning architectures, models, and tips for TensorFlow and PyTorch in Jupyter Notebooks.
- Author: Sebastian Raschka
- GitHub Repository: https://github.com/rasbt/deeplearning-models

In [1]:
%load_ext watermark
%watermark -a 'Sebastian Raschka' -v -p torch

Sebastian Raschka 

CPython 3.6.8
IPython 7.2.0

torch 1.1.0


- Runs on CPU or GPU (if available)

# Model Zoo -- Ordinal Regression CNN -- Niu et al. 2016

Implementation of a method for ordinal regression by Niu et al. [1] applied to predicting age from face images in the AFAD [1] (Asian Face) dataset using a simple ResNet34 [2] convolutional network architecture.

Note that in order to reduce training time, only a subset of AFAD (AFAD-Lite) is being used.

- [1] Niu, Zhenxing, Mo Zhou, Le Wang, Xinbo Gao, and Gang Hua. "[Ordinal regression with multiple output cnn for age estimation](https://ieeexplore.ieee.org/document/7780901/)." In Proceedings of the IEEE conference on computer vision and pattern recognition, pp. 4920-4928. 2016.
- [2] He, Kaiming, Xiangyu Zhang, Shaoqing Ren, and Jian Sun. "[Deep residual learning for image recognition](http://openaccess.thecvf.com/content_cvpr_2016/html/He_Deep_Residual_Learning_CVPR_2016_paper.html)." In Proceedings of the IEEE conference on computer vision and pattern recognition, pp. 770-778. 2016.


## Imports

In [2]:
import time
import numpy as np
import pandas as pd
import os

import torch.nn as nn
import torch.nn.functional as F
import torch

from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torchvision import transforms
from PIL import Image



if torch.cuda.is_available():
    torch.backends.cudnn.deterministic = True

## Downloading the Dataset

In [3]:
!git clone https://github.com/afad-dataset/tarball-lite.git

Cloning into 'tarball-lite'...
remote: Enumerating objects: 37, done.
remote: Total 37 (delta 0), reused 0 (delta 0), pack-reused 37
Unpacking objects: 100% (37/37), done.
Checking out files: 100% (30/30), done.


In [4]:
!cat tarball-lite/AFAD-Lite.tar.xz* > tarball-lite/AFAD-Lite.tar.xz

In [5]:
!tar xf tarball-lite/AFAD-Lite.tar.xz

In [6]:
rootDir = 'AFAD-Lite'

files = [os.path.relpath(os.path.join(dirpath, file), rootDir)
         for (dirpath, dirnames, filenames) in os.walk(rootDir) 
         for file in filenames if file.endswith('.jpg')]

In [7]:
len(files)

59344

In [8]:
d = {}

d['age'] = []
d['gender'] = []
d['file'] = []
d['path'] = []

for f in files:
    age, gender, fname = f.split('/')
    if gender == '111':
        gender = 'male'
    else:
        gender = 'female'
        
    d['age'].append(age)
    d['gender'].append(gender)
    d['file'].append(fname)
    d['path'].append(f)

In [9]:
df = pd.DataFrame.from_dict(d)
df.head()

,age,gender,file,path
0,39,female,474596-0.jpg,39/112/474596-0.jpg
1,39,female,397477-0.jpg,39/112/397477-0.jpg
2,39,female,576466-0.jpg,39/112/576466-0.jpg
3,39,female,399405-0.jpg,39/112/399405-0.jpg
4,39,female,410524-0.jpg,39/112/410524-0.jpg


In [10]:
df['age'].min()

'18'

In [11]:
df['age'] = df['age'].values.astype(int) - 18

In [12]:
np.random.seed(123)
msk = np.random.rand(len(df)) < 0.8
df_train = df[msk]
df_test = df[~msk]

In [13]:
df_train.set_index('file', inplace=True)
df_train.to_csv('training_set_lite.csv')

In [14]:
df_test.set_index('file', inplace=True)
df_test.to_csv('test_set_lite.csv')

In [15]:
num_ages = np.unique(df['age'].values).shape[0]
print(num_ages)

22


## Settings

In [16]:
##########################
### SETTINGS
##########################

# Device
DEVICE = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")

NUM_WORKERS = 8

NUM_CLASSES = 22
BATCH_SIZE = 512
NUM_EPOCHS = 150
LEARNING_RATE = 0.0005
RANDOM_SEED = 123

TRAIN_CSV_PATH = 'training_set_lite.csv'
TEST_CSV_PATH = 'test_set_lite.csv'
IMAGE_PATH = 'AFAD-Lite'

## Dataset Loaders

In [17]:
class AFADDatasetAge(Dataset):
    """Custom Dataset for loading AFAD face images"""

    def __init__(self, csv_path, img_dir, transform=None):

        df = pd.read_csv(csv_path, index_col=0)
        self.img_dir = img_dir
        self.csv_path = csv_path
        self.img_paths = df['path']
        self.y = df['age'].values
        self.transform = transform

    def __getitem__(self, index):
        img = Image.open(os.path.join(self.img_dir,
                                      self.img_paths[index]))

        if self.transform is not None:
            img = self.transform(img)

        label = self.y[index]
        levels = [1]*label + [0]*(NUM_CLASSES - 1 - label)
        levels = torch.tensor(levels, dtype=torch.float32)

        return img, label, levels

    def __len__(self):
        return self.y.shape[0]


custom_transform = transforms.Compose([transforms.Resize((128, 128)),
                                       transforms.RandomCrop((120, 120)),
                                       transforms.ToTensor()])

train_dataset = AFADDatasetAge(csv_path=TRAIN_CSV_PATH,
                               img_dir=IMAGE_PATH,
                               transform=custom_transform)


custom_transform2 = transforms.Compose([transforms.Resize((128, 128)),
                                        transforms.CenterCrop((120, 120)),
                                        transforms.ToTensor()])

test_dataset = AFADDatasetAge(csv_path=TEST_CSV_PATH,
                              img_dir=IMAGE_PATH,
                              transform=custom_transform2)


train_loader = DataLoader(dataset=train_dataset,
                          batch_size=BATCH_SIZE,
                          shuffle=True,
                          num_workers=NUM_WORKERS)

test_loader = DataLoader(dataset=test_dataset,
                         batch_size=BATCH_SIZE,
                         shuffle=False,
                         num_workers=NUM_WORKERS)

## Model

In [18]:
##########################
# MODEL
##########################

class AlexNet(nn.Module):

    def __init__(self, num_classes):
        super(AlexNet, self).__init__()
        self.num_classes = num_classes
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=11, stride=4, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(64, 192, kernel_size=5, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(192, 384, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(384, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
        )
        self.avgpool = nn.AdaptiveAvgPool2d((6, 6))
        self.classifier = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(256 * 6 * 6, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
        )

        self.fc = nn.Linear(4096, (self.num_classes-1)*2)

    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = x.view(x.size(0), 256 * 6 * 6)
        x = self.classifier(x)

        logits = self.fc(x)
        logits = logits.view(-1, (self.num_classes-1), 2)
        probas = F.softmax(logits, dim=2)[:, :, 1]
        return logits, probas


In [19]:
###########################################
# Initialize Cost, Model, and Optimizer
###########################################

def cost_fn(logits, levels):
    val = (-torch.sum((F.log_softmax(logits, dim=2)[:, :, 1]*levels
                      + F.log_softmax(logits, dim=2)[:, :, 0]*(1-levels)), dim=1))
    return torch.mean(val)


torch.manual_seed(RANDOM_SEED)
torch.cuda.manual_seed(RANDOM_SEED)
model = AlexNet(NUM_CLASSES)

model.to(DEVICE)
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

## Training

In [20]:
def compute_mae_and_mse(model, data_loader, device):
    mae, mse, num_examples = 0, 0, 0
    for i, (features, targets, levels) in enumerate(data_loader):

        features = features.to(device)
        targets = targets.to(device)

        logits, probas = model(features)
        predict_levels = probas > 0.5
        predicted_labels = torch.sum(predict_levels, dim=1)
        num_examples += targets.size(0)
        mae += torch.sum(torch.abs(predicted_labels - targets))
        mse += torch.sum((predicted_labels - targets)**2)
    mae = mae.float() / num_examples
    mse = mse.float() / num_examples
    return mae, mse


start_time = time.time()
for epoch in range(NUM_EPOCHS):

    model.train()
    for batch_idx, (features, targets, levels) in enumerate(train_loader):

        features = features.to(DEVICE)
        targets = targets
        targets = targets.to(DEVICE)
        levels = levels.to(DEVICE)

        # FORWARD AND BACK PROP
        logits, probas = model(features)
        cost = cost_fn(logits, levels)
        optimizer.zero_grad()

        cost.backward()

        # UPDATE MODEL PARAMETERS
        optimizer.step()

        # LOGGING
        if not batch_idx % 150:
            s = ('Epoch: %03d/%03d | Batch %04d/%04d | Cost: %.4f'
                 % (epoch+1, NUM_EPOCHS, batch_idx,
                     len(train_dataset)//BATCH_SIZE, cost))
            print(s)

    s = 'Time elapsed: %.2f min' % ((time.time() - start_time)/60)
    print(s)

Epoch: 001/150 | Batch 0000/0092 | Cost: 14.5346
Time elapsed: 0.47 min
Epoch: 002/150 | Batch 0000/0092 | Cost: 10.1990
Time elapsed: 0.93 min
Epoch: 003/150 | Batch 0000/0092 | Cost: 9.3695
Time elapsed: 1.41 min
Epoch: 004/150 | Batch 0000/0092 | Cost: 8.7663
Time elapsed: 1.88 min
Epoch: 005/150 | Batch 0000/0092 | Cost: 8.5821
Time elapsed: 2.36 min
Epoch: 006/150 | Batch 0000/0092 | Cost: 8.8506
Time elapsed: 2.80 min
Epoch: 007/150 | Batch 0000/0092 | Cost: 8.1522
Time elapsed: 3.25 min
Epoch: 008/150 | Batch 0000/0092 | Cost: 9.3045
Time elapsed: 3.74 min
Epoch: 009/150 | Batch 0000/0092 | Cost: 8.2951
Time elapsed: 4.21 min
Epoch: 010/150 | Batch 0000/0092 | Cost: 8.1094
Time elapsed: 4.67 min
Epoch: 011/150 | Batch 0000/0092 | Cost: 8.3870
Time elapsed: 5.15 min
Epoch: 012/150 | Batch 0000/0092 | Cost: 8.1078
Time elapsed: 5.62 min
Epoch: 013/150 | Batch 0000/0092 | Cost: 7.6846
Time elapsed: 6.11 min
Epoch: 014/150 | Batch 0000/0092 | Cost: 7.7015
Time elapsed: 6.56 min
Epoc

## Evaluation

In [21]:
model.eval()
with torch.set_grad_enabled(False):  # save memory during inference

    train_mae, train_mse = compute_mae_and_mse(model, train_loader,
                                               device=DEVICE)
    test_mae, test_mse = compute_mae_and_mse(model, test_loader,
                                             device=DEVICE)

    s = 'MAE/RMSE: | Train: %.2f/%.2f | Test: %.2f/%.2f' % (
        train_mae, torch.sqrt(train_mse), test_mae, torch.sqrt(test_mse))
    print(s)

s = 'Total Training Time: %.2f min' % ((time.time() - start_time)/60)
print(s)

MAE/RMSE: | Train: 0.65/1.13 | Test: 3.91/5.40
Total Training Time: 70.77 min


In [22]:
%watermark -iv

numpy       1.15.4
pandas      0.23.4
torch       1.1.0
PIL.Image   5.3.0

